## Settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import random
import os
import sys
import logging
import torch
import itertools

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
project_root = os.path.abspath('')
if project_root not in sys.path:
    sys.path.append(project_root)
project_root

'/Users/meina/Github/meina-t/matching_with_dl'

In [ ]:
from logging import getLogger
logger = getLogger(__name__)
logger.setLevel(logging.INFO)

sv 
https://arxiv.org/pdf/2107.03427


In [37]:
from model.model import MatchingNet, train_model
from model.params import HParams
from model.data import Data
from model.loss.strategy_proofness import compute_spv
from model.loss.stability import compute_sv
from model.loss.efficiency import generate_stable_matchings, filter_efficient_stable_matchings, compute_ev
from model.utils import is_pareto_dominates, da_with_t

## Train

In [44]:
p_list = list(itertools.permutations([0.5, 0.3333, 0.1666]))
q_list = p_list + [(0.2,0.4,0.4), (0.4,0.2,0.4), (0.4,0.4,0.2),(0.25,.25,0.5), (0.25,0.5,0.25), (0.5,0.25,0.25), (0.3333,0.3333,0.3333)]

In [45]:
pairs_1_1 = []
pairs_1_2 = []
pairs_1_3 = []
pairs_2_1 = []
pairs_2_2 = []
pairs_2_3 = []
pairs_3_1 = []
pairs_3_2 = []
pairs_3_3 = []

p_set_1 = []
p_set_2 = []
p_set_3 = []
q_set_1 = []
q_set_2 = []
q_set_3 = []

for i in range(6):
    p_1 = p_list[i]
    p_set_1.append([p_1 for _ in range(3)])
for i in range(13):
    q_1 = q_list[i]
    q_set_1.append([q_1 for _ in range(3)])

# p_listから重複なく2つ選び、p_2を作成
for i in range(5):
    for j in range(i+1, 6):
        p_set_2.append([p_list[i], p_list[j], p_list[j]])

for i in range(12):
    for j in range(i+1, 13):
        q_set_2.append([q_list[i], q_list[j], q_list[j]])

for i in range(4):
    for j in range(i+1, 5):
        for k in range(j+1, 6):
            p_set_3.append([p_list[i], p_list[j], p_list[k]])

for i in range(11):
    for j in range(i+1, 12):
        for k in range(j+1, 13):
            q_set_3.append([q_list[i], q_list[j], q_list[k]])



for i in p_set_1:
    for j in q_set_1:
        pairs_1_1.append((i, j))

for i in p_set_1:
    for j in q_set_2:
        pairs_1_2.append((i, j))

for i in p_set_1:
    for j in q_set_3:
        pairs_1_3.append((i, j))

for i in p_set_2:
    for j in q_set_1:
        pairs_2_1.append((i, j))

for i in p_set_2:
    for j in q_set_2:
        pairs_2_2.append((i, j))

for i in p_set_2:
    for j in q_set_3:
        pairs_2_3.append((i, j))

for i in p_set_3:
    for j in q_set_1:
        pairs_3_1.append((i, j))

for i in p_set_3:
    for j in q_set_2:
        pairs_3_2.append((i, j))

for i in p_set_3:
    for j in q_set_3:
        pairs_3_3.append((i, j))

In [20]:
p_list = []
q_list = []

for pair in pairs_1_1:
    p_list.append(pair[0])
    q_list.append(pair[1])

df_1_1 = pd.DataFrame({'p': p_list, 'q': q_list})


In [8]:
cfg = HParams(
    num_agents=3,
    num_hidden_nodes=64,
    batch_size=128,
    epochs=2000,
    corr = 0,
    device='mps',
    lr=0.05,
    )

model = MatchingNet(cfg)

In [30]:
data = Data(cfg)

/Users/meina/Github/meina-t/matching_with_dl/model/data.py:60: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  return torch.tensor(matchings, dtype=torch.float32, device=self.device)


In [9]:
device = 'mps'

p = torch.tensor([[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]],  dtype=torch.float32).to(device)
q = torch.tensor([[[1.0, 0.0, 0.0], [0.0,1.0, 0.0], [0.0, 0.0, 1.0]]],  dtype=torch.float32).to(device)

In [57]:
def compute_matching(pairs):
    p_list = []
    q_list = []

    for pair in pairs:
        p_list.append(pair[0])
        q_list.append(pair[1])

    df_1_1 = pd.DataFrame({'p': p_list, 'q': q_list})

    matching_list = []
    spv_list = []
    sv_list = []
    ev_list = []
    da_matching_list = []
    da_spv_list = []
    da_sv_list = []
    da_ev_list = []

    for index, row in df_1_1.iterrows():
        p = torch.tensor(row['p'], dtype=torch.float32, device = 'mps') # torch.tensorに変換
        q = torch.tensor(row['q'], dtype=torch.float32, device = 'mps') # torch.tensorに変換
        p = p.unsqueeze(0) 
        q = q.unsqueeze(0) 

        #matching = model(p, q) 
        #spv = compute_spv(cfg, model, matching, p, q)
        #sv = compute_sv(cfg,matching, p, q)
        #ev = compute_ev(cfg, matching, p, q, data)

        da_matching = da_with_t(p, q)
        da_spv = compute_spv(cfg, da_with_t, da_matching, p, q)
        da_sv = compute_sv(cfg, da_matching, p, q)
        da_ev = compute_ev(cfg, da_matching, p, q, data)

        #matching_list.append(matching.cpu().squeeze(0).tolist()) 
        #spv_list.append(spv.sum().cpu().item())
        #sv_list.append(sv.sum().cpu().item())
        #ev_list.append(ev.sum().cpu().item())

        da_matching_list.append(da_matching.cpu().squeeze(0).tolist())
        da_spv_list.append(da_spv.sum().cpu().item())
        da_sv_list.append(da_sv.sum().cpu().item())
        da_ev_list.append(da_ev.sum().cpu().item())

    #df_1_1['matching'] = matching_list
    #df_1_1['spv'] = spv_list
    #df_1_1['sv'] = sv_list
    #df_1_1['ev'] = ev_list
    df_1_1['da_matching'] = da_matching_list
    df_1_1['da_spv'] = da_spv_list
    df_1_1['da_sv'] = da_sv_list
    df_1_1['da_ev'] = da_ev_list

    return df_1_1

In [58]:
df_1_1 = compute_matching(pairs_1_1)
print('finish 1_1')
df_1_2 = compute_matching(pairs_1_2)
print('finish 1_2')
df_1_3 = compute_matching(pairs_1_3)
print('finish 1_3')
df_2_1 = compute_matching(pairs_2_1)
print('finish 2_1')
df_2_2 = compute_matching(pairs_2_2)
print('finish 2_2')
df_2_3 = compute_matching(pairs_2_3)
print('finish 2_3')
df_3_1 = compute_matching(pairs_3_1)
print('finish 3_1')
df_3_2 = compute_matching(pairs_3_2)
print('finish 3_2')
df_3_3 = compute_matching(pairs_3_3)
print('finish 3_3')


finish 1_1
finish 1_2
finish 1_3
finish 2_1
finish 2_2
finish 2_3
finish 3_1
finish 3_2
finish 3_3


In [61]:
model = MatchingNet(cfg)
model.load_state_dict(torch.load('model_0120.pth'))
model.to(device)
model.eval()

/var/folders/3s/sh8zstl54dl_t2y55d7th57m0000gn/T/ipykernel_34642/395456298.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_0120.p

MatchingNet(
  (layers): Sequential(
    (0): Linear(in_features=18, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=64, out_features=9, bias=True)
  )
)

In [66]:
def add_modeldata(df):
    matching_list = []
    spv_list = []
    sv_list = []
    ev_list = []

    for index, row in df.iterrows():
        p = torch.tensor(row['p'], dtype=torch.float32, device = 'mps') # torch.tensorに変換
        q = torch.tensor(row['q'], dtype=torch.float32, device = 'mps') # torch.tensorに変換
        p = p.unsqueeze(0) 
        q = q.unsqueeze(0) 

        matching = model(p, q) 
        spv = compute_spv(cfg, model, matching, p, q)
        sv = compute_sv(cfg,matching, p, q)
        ev = compute_ev(cfg, matching, p, q, data)

        matching_list.append(matching.cpu().squeeze(0).tolist()) 
        spv_list.append(spv.sum().cpu().item())
        sv_list.append(sv.sum().cpu().item())
        ev_list.append(ev.sum().cpu().item())

    df['matching'] = matching_list
    df['spv'] = spv_list
    df['sv'] = sv_list
    df['ev'] = ev_list

    return df


In [67]:
df_1_1 = add_modeldata(df_1_1)
df_1_2 = add_modeldata(df_1_2)
df_1_3 = add_modeldata(df_1_3)
df_2_1 = add_modeldata(df_2_1)
df_2_2 = add_modeldata(df_2_2)
df_2_3 = add_modeldata(df_2_3)
df_3_1 = add_modeldata(df_3_1)
df_3_2 = add_modeldata(df_3_2)
df_3_3 = add_modeldata(df_3_3)


In [62]:
model(p, q)

tensor([[[2.0241e-01, 5.7346e-04, 7.9702e-01],
         [7.1915e-01, 2.4288e-01, 3.7965e-02],
         [7.8577e-02, 7.5729e-01, 1.6414e-01]]], device='mps:0',
       grad_fn=<DivBackward0>)

In [72]:
import pandas as pd

excel_file = 'output_0120.xlsx' 

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    df_1_1.to_excel(writer, sheet_name='Sheet1_1', index=False)
    df_1_2.to_excel(writer, sheet_name='Sheet1_2', index=False)
    df_1_3.to_excel(writer, sheet_name='Sheet1_3', index=False)
    df_2_1.to_excel(writer, sheet_name='Sheet2_1', index=False)
    df_2_2.to_excel(writer, sheet_name='Sheet2_2', index=False)
    df_2_3.to_excel(writer, sheet_name='Sheet2_3', index=False)
    df_3_1.to_excel(writer, sheet_name='Sheet3_1', index=False)
    df_3_2.to_excel(writer, sheet_name='Sheet3_2', index=False)
    df_3_3.to_excel(writer, sheet_name='Sheet3_3', index=False)
